In [1]:
import os

In [2]:
%%bash
PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-04-014eea1283f0


In [3]:
# TODO: Change these to try this notebook out
PROJECT = "qwiklabs-gcp-04-014eea1283f0"  # Replace with your PROJECT
BUCKET = "chest-xray-us-central"   # defaults to PROJECT
REGION = "us-central1"  # Replace with your REGION

In [4]:
os.environ["PROJECT"] = PROJECT
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.1"
os.environ["PYTHONVERSION"] = "3.7"

In [5]:
%%bash
gcloud config set project ${PROJECT}
gcloud config set compute/region ${REGION}

Updated property [core/project].
Updated property [compute/region].


In [6]:
%%bash
gsutil ls gs://${BUCKET}/pneumonia/trained_model

gs://chest-xray-us-central/pneumonia/trained_model/
gs://chest-xray-us-central/pneumonia/trained_model/20210624043339/
gs://chest-xray-us-central/pneumonia/trained_model/20210624063522/


In [10]:
%%bash
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/pneumonia/trained_model/2* \
                 | tail -1)
gsutil ls ${MODEL_LOCATION}

gs://chest-xray-us-central/pneumonia/trained_model/20210624063522/
gs://chest-xray-us-central/pneumonia/trained_model/20210624063522/saved_model.pb
gs://chest-xray-us-central/pneumonia/trained_model/20210624063522/assets/
gs://chest-xray-us-central/pneumonia/trained_model/20210624063522/variables/


In [17]:
# %%writefile input/predict.py
import os
import shutil
import datetime

import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

def process_path(path_file):
    image = tf.io.read_file(path_file)    
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)

    image = tf.image.resize(image, [127, 127], method='area')
    print(image)
    return image

def reader_image(path_file):

    list_ds = tf.data.Dataset.list_files(path_file)
    labeled_ds = list_ds.map(process_path(path_file))

    return labeled_ds  
 
    
testdata = reader_image("gs://chest-xray-us-central/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg")
print("hello")
print(testdata)

tf.Tensor(
[[[0.14625284 0.14625284 0.14625284]
  [0.1141312  0.1141312  0.1141312 ]
  [0.09546831 0.09546831 0.09546831]
  ...
  [0.35582545 0.35582545 0.35582545]
  [0.3127868  0.3127868  0.3127868 ]
  [0.2593304  0.2593304  0.2593304 ]]

 [[0.11699042 0.11699042 0.11699042]
  [0.07214279 0.07214279 0.07214279]
  [0.08080833 0.08080833 0.08080833]
  ...
  [0.33638778 0.33638778 0.33638778]
  [0.29033482 0.29033482 0.29033482]
  [0.2441124  0.2441124  0.2441124 ]]

 [[0.00529208 0.00529208 0.00529208]
  [0.03266406 0.03266406 0.03266406]
  [0.06660769 0.06660769 0.06660769]
  ...
  [0.32328027 0.32328027 0.32328027]
  [0.28213194 0.28213194 0.28213194]
  [0.22785257 0.22785257 0.22785257]]

 ...

 [[0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  ...
  [0.         0.         0.        ]
  [0.         0.         0.        ]
  [0.         0.         0.        ]]

 [[0.         0.         0.        ]
  [0.         0.         0

ValueError: Argument must be callable

In [12]:
import tensorflow as tf
print(tf.__version__)

2.6.0-dev20210621


In [14]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "pneumonia"
MODEL_VERSION = "v1"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict" \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {"Authorization": "Bearer " + token }
data = {
  "instances": [
 {
    "gs://chest-xray-us-central/chest_xray/val/NORMAL/NORMAL2-IM-1427-0001.jpeg"
 }
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

TypeError: Object of type set is not JSON serializable